In [3]:
import pandas as pd
from bs4 import BeautifulSoup

In [5]:
from bs4 import BeautifulSoup
import requests
from os.path import basename

In [28]:
from webbot import Browser 

url = 'https://anchor.fm/dashboard'
username = 'cherylschwahn@icloud.com'
password = 'Sean1998!'

web = Browser()
web.go_to(url) 

web.type(username , into='Email')
web.type(password , into='Password')

web.click('Log In')

web.click('No thanks')

# web.click('Weekly')

# web.click('Daily')

# web.click('Export current view as CSV')

# TODO: demographics - m/f/d share and age

In [9]:
# get html
requests_get = requests.get(url).text
soup = BeautifulSoup(requests_get)

In [25]:
# unix timestamp
date_start = 1602115200 # 2020-10-08
interval_weekly = 604800
interval_daily = 

from datetime import datetime


# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(date_start).strftime('%Y-%m-%d %H:%M:%S'))

2020-10-08 00:00:00


In [30]:
# url does not seem to change for different logins
weekly = "https://anchor.fm/api/proxy/v3/analytics/station/webStationId:36f5b4ec/plays?timeInterval=604800&timeRangeStart=1602115200&timeRangeEnd=1610582400&csvFilename=RWYToSustainability_TotalPlays_2020-10-08_to_2021-01-14.csv"
daily = "https://anchor.fm/api/proxy/v3/analytics/station/webStationId:36f5b4ec/plays?timeInterval=86400&timeRangeStart=1602115200&timeRangeEnd=1610582400&csvFilename=b-10-08_to_2021-01-14.csv"

In [38]:
req = requests.get(weekly)
url_content = req.content
csv_file = open('downloaded.csv', 'wb')

csv_file.write(url_content)
csv_file.close()

In [32]:
for link in soup.find_all(text='Download CSV'): 
    print(link)

# Dump

In [ ]:
def webbot_init(url):
    """Initialize webbot, return browser object"""
    # init webbot browser
    web = Browser()
    web.go_to(url)
    
    return web

def wait_until(somepredicate, timeout, period=0.25, *args, **kwargs):
    """Wait until condition is satisfied """
    mustend = time() + timeout
    while time() < mustend:
        if somepredicate: return True
        sleep(period)
    return False

def login(webbrowser, url, pw, email):
    """Login with webbot in URL"""   
    # wait until login fiels exist
    login_exists = webbrowser.exists(text='Email or Username', id='Input_UserName', loose_match=False)
    wait_until(login_exists, timeout=5)
    
    # login with username and password
    webbrowser.type(email, into='Email or Username')#, id="Input_UserName")
    webbrowser.type(pw, into='Password')#, id="Input_Password") 
    
    # click sign in button
    webbrowser.click('SIGN IN')
    
def get_html(webbrowser):
    """Obtain HTML source code of current page"""
    # get page content
    html = webbrowser.get_page_source() # get source code
    soup = BeautifulSoup(html) # parse HTML with BS
    
    return soup



# credentials
login_url = "https://app.wodify.com/SignIn/Login?OriginalURL=&RequiresConfirm=false"
pw = 'Papasierra1'
email = 'philippschmalen@gmail.com'
# main wod page
wod_url = 'https://app.wodify.com/WOD/WODEntry.aspx'

# init webbrowser
webbrowser = webbot_init(login_url)
# login to page
login(webbrowser, login_url, pw, email)

## Scrape WODs

In [ ]:
## sepcify time window
years = 3
days_backwards = years*365
days_backwards_continued = (years+2)*365

for t in range(days_backwards,days_backwards+days_backwards_continued): #changed range to 
    day = date.today()-timedelta(days=t)
    print(day)
    webbrowser.type(day.strftime("%d-%m-%Y"), id='AthleteTheme_wtLayoutNormal_block_wtSubNavigation_wttxtDate') #into=date.today().strftime("%Y-%m-%d"))
    
    # press enter
    webbrowser.press(webbrowser.Key.ENTER)
    
    # wait a bit
    sleep(1.6)
    
    # get html source code
    html = webbrowser.get_page_source() # get source code
    soup = BeautifulSoup(html) # parse HTML with BS
    # get text
    text_all = soup.get_text(separator="\n")

    ## isolate text body
    # body is enclosed by '\xa0\n' and '\n\xa0'
    # replace those parts with # to split the string
    text_all = text_all.replace("\xa0\n", '#').replace("\n\xa0", '#')
    text_body1 = text_all.split("#")[2]
    text_body2 = text_all.split("#")[3]

    # select section titles with div class=''
    section_titles = [section_title.text for section_title in soup.find_all("div", class_="section_title")]
    # get workout text
    try:
        workout1 = re.split("(\{})".format(section_titles[0]), text_body1)
        workout2 = re.split("(\{})".format(section_titles[0]), text_body2)
    except:
        workout1 = 'failed'
        workout2 = 'failed'
    
    # store page content in DataFrame as row 
    row = pd.DataFrame([day, section_titles, workout1, workout2]).T
    
    # export to CSV
    row.to_csv('../data/raw/wod_content_years_3_to_5.csv', 
              header=False, 
              index=False,
               mode='a')